# Hidden Markov Model in TensorFlow
## Implementation of Viterbi & Forward-Backward Algorithms

In [8]:
import tensorflow as tf
import numpy as np
from __future__ import print_function

In [9]:
def dptable(state_prob):
    print(" ".join(("%10d" % i) for i in range(state_prob.shape[0])))
    for i, prob in enumerate(state_prob.T):
        print("%.7s: " % states[i] +" ".join("%.7s" % ("%f" % p) for p in prob))

In [10]:
from HiddenMarkovModel import HiddenMarkovModel

## Viterbi Example

In [11]:
p0 = np.array([0.6, 0.4])

emi = np.array([[0.5, 0.1],
                [0.4, 0.3],
                [0.1, 0.6]])

trans = np.array([[0.7, 0.3],
                  [0.4, 0.6]])

states = {0:'Healthy', 1:'Fever'}
obs = {0:'normal', 1:'cold', 2:'dizzy'}

obs_seq = np.array([0, 1, 2])

In [12]:
model =  HiddenMarkovModel(trans, emi, p0)
states_seq, state_prob = model.run_viterbi(obs_seq)
dptable(state_prob)
print()
print("Most likely States: ", [states[s] for s in states_seq])

         0          1          2
Healthy: 0.30000 0.08400 0.00588
Fever: 0.04000 0.02700 0.01512

Most likely States:  ['Healthy', 'Healthy', 'Fever']


# Forward - Backward Example

In [13]:
p0 = np.array([0.5, 0.5])

emi = np.array([[0.9, 0.2],
                [0.1, 0.8]])

trans = np.array([[0.7, 0.3],
                  [0.3, 0.7]])

states = {0:'rain', 1:'no_rain'}
obs = {0:'umbrella', 1:'no_umbrella'}

obs_seq = np.array([0, 0, 1, 0, 0])

In [14]:
model =  HiddenMarkovModel(trans, emi, p0)

results = model.run_forward_backward(obs_seq)
result_list = ["Forward", "Backward", "Posterior"]

for state_prob, path in zip(results, result_list) :
    inferred_states = np.argmax(state_prob, axis=1)
    print()
    print(path)
    print("Most likely States: ",[states[s] for s in inferred_states])
    print()
    dptable(state_prob)
    print()


Forward
Most likely States:  ['rain', 'rain', 'no_rain', 'rain', 'rain']

         0          1          2          3          4
rain: 0.81818 0.88335 0.19066 0.73079 0.86733
no_rain: 0.18181 0.11664 0.80933 0.26920 0.13266


Backward
Most likely States:  ['rain', 'rain', 'no_rain', 'rain', 'rain']

         0          1          2          3          4
rain: 0.64693 0.59231 0.37626 0.65334 0.62727
no_rain: 0.35306 0.40768 0.62373 0.34665 0.37272


Posterior
Most likely States:  ['rain', 'rain', 'no_rain', 'rain', 'rain']

         0          1          2          3          4
rain: 0.89184 0.91668 0.12443 0.83650 0.91668
no_rain: 0.10816 0.08331 0.87556 0.16349 0.08331

